In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)
from scipy.stats import mode
from scipy import stats
import feather
import re
import warnings
import random

from IPython.display import clear_output

In [107]:
target = pd.read_csv('target2',header=None,index_col=0)

In [3]:
train = feather.read_dataframe('best_table\\train')
test = feather.read_dataframe('best_table\\test')

In [5]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + [x for x in features if re.search('[th]_cat1_[^s]',x)]

In [8]:
warnings.filterwarnings("ignore")
param = {
        'num_leaves': 111, #checked
         'num_threads': 4,
         'max_bin': 170, #checked 170
         'min_data_in_leaf': 100, #checked 100
         'objective':'regression',
         'max_depth': 9, #checked
         'tree_learner':'data',
         'learning_rate': 0.07,
         "boosting": "gbdt", #dart, gbdt
         "feature_fraction": 0.9, #checked
         "bagging_freq": 5, #5
         "bagging_fraction": 0.9, #checked
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.26, #checked 26
#            'lambda_l2': 0.3, #checked
         "random_state": 77,
         "verbosity": -1,
         "device":"gpu",
         'gpu_device_id':0,
         'gpu_platform_id':1,
         'gpu_use_dp':True,
        }


folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions1 = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))

    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=categorical_feats
                          )


    clf = lgb.train(param, trn_data,10000, valid_sets = [trn_data, val_data], verbose_eval=250, early_stopping_rounds = 200)

    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions1 += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
errr = mean_squared_error(oof, target)**0.5
print("CV score: {:<8.5f}".format(errr))

fold n°0


LightGBMError: Multiclass objective and metrics don't match

In [6]:
train['target'] = target
train = train[train.target > -30]
target = train.pop('target')

In [7]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + [x for x in features if re.search('[th]_cat1_[^s]',x)]

In [8]:
warnings.filterwarnings("ignore")
param = {
        'num_leaves': 111, #checked
         'num_threads': 4,
         'max_bin': 170, #checked 170
         'min_data_in_leaf': 100, #checked 100
         'objective':'regression',
         'max_depth': 9, #checked
         'tree_learner':'data',
         'learning_rate': 0.0003,
         "boosting": "gbdt", #dart, gbdt
         "feature_fraction": 0.9, #checked
         "bagging_freq": 5, #5
         "bagging_fraction": 0.9, #checked
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.26, #checked 26
#            'lambda_l2': 0.3, #checked
         "random_state": 77,
         "verbosity": -1,
         "device":"gpu",
         'gpu_device_id':0,
         'gpu_platform_id':1,
         'gpu_use_dp':True,
        }


folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions2 = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))

    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=categorical_feats
                          )


    clf = lgb.train(param, trn_data,100000, valid_sets = [trn_data, val_data], verbose_eval=2500, early_stopping_rounds = 2000)

    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions2 += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
errr = mean_squared_error(oof, target)**0.5
print("CV score: {:<8.5f}".format(errr))

fold n°0
Training until validation scores don't improve for 2000 rounds.
[2500]	training's rmse: 1.59995	valid_1's rmse: 1.6185
[5000]	training's rmse: 1.55421	valid_1's rmse: 1.58631
[7500]	training's rmse: 1.52938	valid_1's rmse: 1.57379
[10000]	training's rmse: 1.51148	valid_1's rmse: 1.56765
[12500]	training's rmse: 1.49682	valid_1's rmse: 1.56415
[15000]	training's rmse: 1.48447	valid_1's rmse: 1.56207
[17500]	training's rmse: 1.47351	valid_1's rmse: 1.56078
[20000]	training's rmse: 1.46316	valid_1's rmse: 1.55992
[22500]	training's rmse: 1.45355	valid_1's rmse: 1.55932
[25000]	training's rmse: 1.44426	valid_1's rmse: 1.55892
[27500]	training's rmse: 1.43531	valid_1's rmse: 1.55864
[30000]	training's rmse: 1.42662	valid_1's rmse: 1.55844
[32500]	training's rmse: 1.41831	valid_1's rmse: 1.55827
[35000]	training's rmse: 1.41021	valid_1's rmse: 1.55819
[37500]	training's rmse: 1.40233	valid_1's rmse: 1.55814
[40000]	training's rmse: 1.39458	valid_1's rmse: 1.55809
Early stopping, bes

In [9]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions1
sub_df.to_csv("submit1_with_outs.csv", index=False)

In [10]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions2
sub_df.to_csv("submit2_without_outs.csv", index=False)

In [ ]:
################OUTLINERS TESTING################

In [4]:
target = target < -30

In [6]:
features = [c for c in train.columns if c not in ['card_id', 'first_active_month','last_active_month']]
categorical_feats = [x for x in features if (re.search('modeplus',x) or re.search('feature_',x))] + [x for x in features if re.search('[th]_cat1_[^s]',x)]

In [13]:
warnings.filterwarnings("ignore")
param = {
    'num_leaves': 111, #checked
     'num_threads': 4,
     'max_bin': 170, #checked 170
     'min_data_in_leaf': 100, #checked 100
     'objective':'binary',
     'max_depth': 9, #checked
     'tree_learner':'data',
     'learning_rate': 0.0003,
     "boosting": "gbdt", #dart, gbdt
     "feature_fraction": 0.58, #checked 0.58
     "bagging_freq": 7, #5
     "bagging_fraction": 0.97, #checked
     "bagging_seed": 11,
     "metric": 'binary',
     "lambda_l1": 0.23, #checked 26
#            'lambda_l2': 0.3, #checked
     "random_state": 77,
     "verbosity": -1,
     "device":"gpu",
     'gpu_device_id':0,
     'gpu_platform_id':1,
     'gpu_use_dp':True,
    }


folds = KFold(n_splits=5, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions2 = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))

    trn_data = lgb.Dataset(train.iloc[trn_idx][features],
                           label=target.iloc[trn_idx],
                           categorical_feature=categorical_feats
                          )
    val_data = lgb.Dataset(train.iloc[val_idx][features],
                           label=target.iloc[val_idx],
                           categorical_feature=categorical_feats
                          )


    clf = lgb.train(param, trn_data,100000, valid_sets = [trn_data, val_data], verbose_eval=2500, early_stopping_rounds = 2000)

    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)

    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    predictions2 += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
    
errr = mean_squared_error(oof, target)**0.5
print("CV score: {:<8.5f}".format(errr))

fold n°0
Training until validation scores don't improve for 2000 rounds.
[2500]	training's binary_logloss: 0.038984	valid_1's binary_logloss: 0.0482518
[5000]	training's binary_logloss: 0.0323061	valid_1's binary_logloss: 0.0458645
[7500]	training's binary_logloss: 0.0278489	valid_1's binary_logloss: 0.0450832
[10000]	training's binary_logloss: 0.0248229	valid_1's binary_logloss: 0.0448194
[12500]	training's binary_logloss: 0.0225898	valid_1's binary_logloss: 0.0447876
Early stopping, best iteration is:
[11956]	training's binary_logloss: 0.0230349	valid_1's binary_logloss: 0.044781
fold n°1
Training until validation scores don't improve for 2000 rounds.
[2500]	training's binary_logloss: 0.03967	valid_1's binary_logloss: 0.0467697
[5000]	training's binary_logloss: 0.0328908	valid_1's binary_logloss: 0.0444508
[7500]	training's binary_logloss: 0.028355	valid_1's binary_logloss: 0.043678
[10000]	training's binary_logloss: 0.0251003	valid_1's binary_logloss: 0.0433947
[12500]	training's bi

In [38]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions2
sub_df.to_csv("binary_outliner_chance.csv", index=False)

In [53]:
sub_df = sub_df.sort_values(by='target',ascending=False).reset_index(drop=True)

In [55]:
sub_df['target'] = (sub_df.index < 1800).astype(np.float64)

In [58]:
best_with = pd.read_csv('submit1_with_outs.csv')
best_witho = pd.read_csv("submit2_without_outs.csv")

In [62]:
best_with.columns = ['card_id','target_with']
best_witho.columns = ['card_id','target_witho']

In [66]:
combo = pd.merge(sub_df,best_with,on='card_id',how='left')
combo = pd.merge(combo,best_witho,on='card_id',how='left')

In [69]:
combo['difference'] = np.abs(combo.target_with - combo.target_witho)

In [76]:
end_combo = pd.DataFrame()

In [77]:
end_combo['card_id'] = combo['card_id']

In [109]:
end_combo['target'] = ((combo.difference) >= 9) * -33.219281
end_combo['target'] += ((combo.difference < 9) & (combo.difference > 2.0)) * combo.target_with
end_combo['target'] += ((combo.difference) <= 2) * combo.target_witho

In [112]:
end_combo['target'] += (combo.difference > 1.5) * combo.target_with
end_combo['target'] += ((combo.difference) <= 1.5) * combo.target_witho

In [113]:
end_combo.to_csv('full_combination_std_check.csv', index=False)

In [103]:
combo.target_with.min()

-3.626209400461154

In [108]:
target.min()

1   -33.219281
dtype: float64